In [45]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import torch.functional as F
import torch.utils.data as data
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [46]:
df = pd.read_csv("train_data.csv")

In [47]:
df

,SalePrice,YearBuilt,Size(sqf),Floor,HallwayType,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_manager,N_elevators,SubwayStation,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,141592,2006,814,3,terraced,individual_heating,management_in_trust,111.0,184.0,5min~10min,10min~15min,3.0,0.0,Kyungbuk_uni_hospital,5,6.0,9.0
1,51327,1985,587,8,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
2,48672,1985,587,6,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
3,380530,2006,2056,8,terraced,individual_heating,management_in_trust,249.0,536.0,0~5min,0-5min,5.0,11.0,Sin-nam,5,3.0,7.0
4,78318,1992,644,2,mixed,individual_heating,self_management,142.0,79.0,5min~10min,15min~20min,4.0,8.0,Myung-duk,3,9.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,570796,2007,1928,24,terraced,individual_heating,management_in_trust,0.0,1270.0,0~5min,0-5min,14.0,16.0,Kyungbuk_uni_hospital,10,9.0,10.0
4120,307079,2015,644,22,terraced,individual_heating,management_in_trust,102.0,400.0,0~5min,5min~10min,5.0,10.0,Daegu,7,7.0,11.0
4121,357522,2007,868,20,terraced,individual_heating,management_in_trust,0.0,1270.0,0~5min,0-5min,14.0,16.0,Kyungbuk_uni_hospital,10,9.0,10.0
4122,312389,1978,1327,1,corridor,individual_heating,self_management,87.0,0.0,0~5min,0-5min,1.0,4.0,Kyungbuk_uni_hospital,3,7.0,11.0


In [48]:
print(len(df.columns))

17


In [49]:
categorical_columns = ["HallwayType", "SubwayStation"]

print(df["TimeToBusStop"].unique())
print(df["TimeToSubway"].unique())

['5min~10min' '0~5min' '10min~15min']
['10min~15min' '5min~10min' '0-5min' '15min~20min' 'no_bus_stop_nearby']


In [50]:
dataType = "TimeToBusStop"
df.loc[df[dataType] == '0~5min', dataType] = np.float64(2.0)
df.loc[df[dataType] == '5min~10min', dataType] = np.float64(1.0)
df.loc[df[dataType] == '10min~15min', dataType] = np.float64(0.0)
print(df[dataType].unique())

dataType = "TimeToSubway"
df.loc[df[dataType] == 'no_bus_stop_nearby', dataType] = np.float64(0.0)
df.loc[df[dataType] == '0-5min', dataType] = np.float64(4.0)
df.loc[df[dataType] == '5min~10min', dataType] = np.float64(3.0)
df.loc[df[dataType] == '10min~15min', dataType] = np.float64(2.0)
df.loc[df[dataType] == '15min~20min', dataType] = np.float64(1.0)
print(df[dataType].unique())

[1.0 2.0 0.0]
[2.0 3.0 4.0 1.0 0.0]


In [51]:
df.HeatingType = (df.HeatingType==df["HeatingType"].unique()[0]).astype(int)
df.AptManageType = (df.AptManageType==df["AptManageType"].unique()[0]).astype(int)

In [52]:
categorical_values0 = pd.get_dummies(df[categorical_columns[0]])
categorical_values0.head()
categorical_values1 = pd.get_dummies(df[categorical_columns[1]])
categorical_values1.head()

,Bangoge,Banwoldang,Chil-sung-market,Daegu,Kyungbuk_uni_hospital,Myung-duk,Sin-nam,no_subway_nearby
0,0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0


In [53]:
df.drop(columns=categorical_columns,inplace=True)
df = df.astype(float)


In [74]:
train_indices = np.random.rand(len(df))>0.3
print(len(train_indices))

4124


In [55]:
df

,SalePrice,YearBuilt,Size(sqf),Floor,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_manager,N_elevators,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,141592.0,2006.0,814.0,3.0,1.0,1.0,111.0,184.0,1.0,2.0,3.0,0.0,5.0,6.0,9.0
1,51327.0,1985.0,587.0,8.0,1.0,0.0,80.0,76.0,2.0,3.0,2.0,2.0,3.0,12.0,4.0
2,48672.0,1985.0,587.0,6.0,1.0,0.0,80.0,76.0,2.0,3.0,2.0,2.0,3.0,12.0,4.0
3,380530.0,2006.0,2056.0,8.0,1.0,1.0,249.0,536.0,2.0,4.0,5.0,11.0,5.0,3.0,7.0
4,78318.0,1992.0,644.0,2.0,1.0,0.0,142.0,79.0,1.0,1.0,4.0,8.0,3.0,9.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,570796.0,2007.0,1928.0,24.0,1.0,1.0,0.0,1270.0,2.0,4.0,14.0,16.0,10.0,9.0,10.0
4120,307079.0,2015.0,644.0,22.0,1.0,1.0,102.0,400.0,2.0,3.0,5.0,10.0,7.0,7.0,11.0
4121,357522.0,2007.0,868.0,20.0,1.0,1.0,0.0,1270.0,2.0,4.0,14.0,16.0,10.0,9.0,10.0
4122,312389.0,1978.0,1327.0,1.0,1.0,0.0,87.0,0.0,2.0,4.0,1.0,4.0,3.0,7.0,11.0


In [56]:
numerical_data = torch.from_numpy(df.values[train_indices,1:]).float()
categorical_data0 = torch.from_numpy(categorical_values0.values[train_indices]).float()
categorical_data1 = torch.from_numpy(categorical_values1.values[train_indices]).float()
targets = torch.from_numpy(df.values[train_indices,0]).float()

test_numerical_data = torch.from_numpy(df.values[~train_indices,1:]).float()
test_categorical_data0 = torch.from_numpy(categorical_values0.values[~train_indices]).float()
test_categorical_data1 = torch.from_numpy(categorical_values1.values[~train_indices]).float()
test_targets = torch.from_numpy(df.values[~train_indices,0]).float()

In [57]:
scaler = StandardScaler()
numerical_data = scaler.fit_transform(numerical_data)
print(numerical_data)
numerical_data=torch.from_numpy(numerical_data).float()

[[ 0.34046258 -0.39445279 -1.17223111 ... -0.36374166 -1.12894405
  -0.43359027]
 [ 0.34046258  2.78299072 -0.52507972 ... -0.36374166 -2.00776093
  -0.89019387]
 [-1.27027377 -0.82936857 -1.30166139 ... -1.22028977 -0.25012716
   0.70791873]
 ...
 [ 0.45551518  2.45552472  1.54580473 ...  1.77762863 -0.25012716
  -0.20528847]
 [ 1.37593596 -0.82936857  1.28694417 ...  0.49280646 -0.83600508
   0.02301333]
 [ 0.45551518 -0.25630307  1.02808361 ...  1.77762863 -0.25012716
  -0.20528847]]


In [58]:
train_dataset = data.TensorDataset(numerical_data,categorical_data0, categorical_data1,targets)
test_dataset = data.TensorDataset(test_numerical_data,test_categorical_data0,test_categorical_data1,test_targets)

In [59]:
num_inputs = df.shape[1] + len(categorical_columns) - 1
print(num_inputs)

16


In [60]:
# class AffordabilityEstim(nn.Module):
#     def __init__(self, num_inputs, num_outputs):
#         super(AffordabilityEstim, self).__init__()
#         self.lin1 =nn.Linear(num_inputs, 48)
#         self.act1 =nn.ReLU()
#         self.lin2 =nn.Linear(48, 24)
#         self.act2 =nn.ReLU()
#         self.lin3 =nn.Linear(24, num_outputs)

#     def forward(self, x, cat_x0, cat_x1):
#         x = torch.cat([x,cat_x0, cat_x1],dim=1)
#         x = self.lin1(x)
#         x = self.bn1(x)
#         x = self.act1(x)
#         x = self.lin2(x)
#         x = self.bn2(x)
#         x = self.act2(x)
#         x = self.lin3(x)
#         return x
    
    # def forward(self, x, cat_x):
        # x = torch.cat([x,cat_x],dim=1)
        # activation1 = self.act_1(self.layer1(x))
        # activation1 = self.d1(activation1)
        # activation2 = self.act_2(self.layer2(activation1))
        # activation2 = self.d1(activation2)
        # output = self.layer3(activation2)
        # return output

In [61]:
class AffordabilityEstim(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(AffordabilityEstim, self).__init__()
        self.lin1 = nn.Linear(df.shape[1] + categorical_data0.shape[1] + categorical_data1.shape[1] -1, 48)
        # self.lin1 = nn.Linear(num_inputs, 48)
        self.act1 = nn.Tanh()
        self.lin2 = nn.Linear(48, 24)
        self.act2 = nn.ReLU()
        self.lin3 = nn.Linear(24, num_outputs)

    def forward(self, x, cat_x0, cat_x1):
        x = torch.cat([x, cat_x0, cat_x1], dim=1)
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        x = self.lin3(x)
        return x

In [62]:
# class AffordabilityEstim(nn.Module):
#     def __init__(self, num_inputs, num_outputs):
#         super(AffordabilityEstim, self).__init__()
#         self.lin1 = nn.Linear(df.shape[1] + categorical_data0.shape[1] + categorical_data1.shape[1] -1, 48)
#         # self.lin1 = nn.Linear(num_inputs, 48)
#         self.act1 = nn.Tanh()
#         self.lin2 = nn.Linear(48, 24)
#         self.act2 = nn.ReLU()
#         self.lin3 = nn.Linear(24, num_outputs)

#     def forward(self, x, cat_x0, cat_x1):
#         x = torch.cat([x, cat_x0, cat_x1], dim=1)
#         x = self.lin1(x)
#         x = self.act1(x)
#         x = self.lin2(x)
#         x = self.act2(x)
#         x = self.lin3(x)
#         return x

In [72]:
class AffordabilityEstimBatchNorm(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(AffordabilityEstimBatchNorm, self).__init__()
        self.lin1 = nn.Linear(df.shape[1] + categorical_data0.shape[1] + categorical_data1.shape[1] -1, 48)
        self.bn1 = nn.BatchNorm1d(48)
        self.act1 =nn.ReLU()
        self.lin2 =nn.Linear(48, 24)
        self.bn2 = nn.BatchNorm1d(24)
        self.act2 =nn.ReLU()
        self.lin3 =nn.Linear(24, num_outputs)


    def forward(self, x, cat_x0, cat_x1):
        x = torch.cat([x, cat_x0, cat_x1], dim=1)
        x = self.lin1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.lin3(x)
        return x



In [64]:
model = AffordabilityEstim(16, 1)
print(model)

AffordabilityEstim(
  (lin1): Linear(in_features=25, out_features=48, bias=True)
  (act1): Tanh()
  (lin2): Linear(in_features=48, out_features=24, bias=True)
  (act2): ReLU()
  (lin3): Linear(in_features=24, out_features=1, bias=True)
)


In [65]:
def classify(x):
    # CHEAP
    if x <= 100000:
        return 0
    # AVERAGE
    if x > 100000 and x <= 350000:
        return 1
    # EXPENSIVE 
    if x > 350000:
        return 2

In [76]:
loss_fn = nn.MSELoss()
# opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [67]:
train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [68]:
def get_accuracy(model, data_loader):
    # ZMIENIĆ TO NA REGRESJĘ
    correct = 0
    total = 0
    model.eval() #*********#
    for x, cat_x0, cat_x1, labels in data_loader:
        output = model(x, cat_x0, cat_x1)
        pred = output>0
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += x.shape[0]
    return correct / total

In [79]:
epochs = 100

# for epoch in range(epochs):
#     for i, batch in enumerate(train_loader, 0):

#         model.train()

#         inputs, labels = batch

#         # training steps for normal model
#         opt.zero_grad()
#         outputs = model(inputs).squeeze()
#         loss = loss_fn(outputs, labels)
#         loss.backward()
#         opt.step()

iters = []
losses = []
train_acc = []
val_acc = []
model = AffordabilityEstimBatchNorm(16, 1)
for n in range(epochs):
    epoch_losses = []
    for x, cat_x0, cat_x1, labels in train_loader:
        model.train()
        out = model(x, cat_x0,cat_x1).squeeze()
        print(out)

        loss = loss_fn(out, labels)
        loss.backward()
        epoch_losses.append(loss.item())
        optimizer.step()
        optimizer.zero_grad()

    loss_mean = np.array(epoch_losses).mean()
    iters.append(n)
    losses.append(loss_mean)
    test_acc = get_accuracy(model, test_loader)
    print(f"Epoch {n} loss {loss_mean:.3} test_acc: {test_acc:.3}")
    train_acc.append(get_accuracy(model, train_loader)) # compute training accuracy
    val_acc.append(test_acc)  # compute validation accuracy


print("Final Training Accuracy: {}".format(train_acc[-1]))
print("Final Validation Accuracy: {}".format(val_acc[-1]))

tensor([ 0.4646,  0.5214,  0.2900,  0.9593,  0.2667,  0.2337, -0.0910, -0.1396,
         0.0283,  0.4749,  0.4566,  0.8201,  0.1175,  0.0404,  0.4022,  0.1823,
         0.1622, -0.1531,  0.2410,  0.1920, -0.1214,  0.2172,  0.3902, -0.0041,
         0.2442, -0.1377,  0.2511,  0.0279,  0.1490,  0.5962,  0.5573,  0.2569,
         0.1823,  0.1280, -0.0790,  0.0943,  0.4920,  0.2505,  0.7733, -0.0091,
        -0.0737, -0.0370, -0.2881,  0.1321,  0.2671,  0.0371,  0.5134, -0.0946,
         0.5328, -0.2292,  0.1962,  0.2728,  0.5742,  0.4148,  0.6967,  0.3253,
        -0.0863,  0.0594,  0.3172,  0.4835,  0.0900,  0.5939,  0.2773,  0.3209],
       grad_fn=<SqueezeBackward0>)
tensor([ 0.3131,  0.4627,  0.2670,  0.5868,  0.1806,  0.4462, -0.0954,  0.3156,
         0.1667,  0.2240,  0.2223,  0.6629,  0.2823, -0.1917,  0.3820,  0.2575,
         0.1535,  0.4352,  0.2611,  0.0673, -0.1914,  0.3449,  0.5541,  0.5122,
         0.1993,  0.1781,  0.1528,  0.1931, -0.1841,  0.0697,  0.3656,  0.3213,
    